In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [2]:
import os
from pathlib import Path

import pandas as pd
import torch
import numpy as np 
import scipy

import plotly.express as px

from eff_dim_solver.datasets.dataloading_utils import load_rcv1_data, load_real_data
from eff_dim_solver.sketching.sketches import sjlt
from eff_dim_solver.solvers.direct_method import DirectMethod
from eff_dim_solver.solvers.conjugate_gradient import CG
from eff_dim_solver.solvers.preconditioned_conjugate_gradient import PCG
from scipy.sparse import identity as sparse_identity_matrix
from scipy.sparse.linalg import factorized as sparse_factorization

from timeit import default_timer as time

ModuleNotFoundError: No module named 'eff_dim_solver'

In [239]:
dataset_name = 'synthetic'
xp_name = '1107_0624'
xp_dir = Path(f'/Users/jonathanlacotte/code/numerical_results/effective_dimension_solver/{dataset_name}/target_tolerance') / xp_name

df = pd.read_parquet(xp_dir)

try:
    df = df.sort_values(['n', 'd', 'cn', 'method', 'sketch_fn'])
except:
    df = df.sort_values(['tolerance', 'reg_param', 'method', 'sketch_fn'])

In [241]:
for d in np.unique(df.d):
    print(f"{d=}")
    df_ = df[(df.d==d)&(df.cn==1024)]
    fig = px.histogram(df_, 
                       x="sketch_fn", 
                       y="time",
                       color="method", 
                       pattern_shape="sketch_fn",
                       barmode='group',
                       text_auto='.2s',
                       facet_row='n',
                       facet_col='deff',
                       #log_y=True,
                      )

    fig.update_traces(textangle=0, textposition="outside", cliponaxis=False)

    fig.update_layout(height=700)
    fig.show()

d=64


d=128


d=256


d=512


d=1024


d=2048


d=4096


In [193]:
df[(df.n==2048)&(df.d==64)&(df.cn==2048)&(df.method=='dm')]

,n,d,deff,cn,reg_param,pcg_sketch_size,ada_sketch_size,max_sketch_size,tolerance,tolerance_warmup,n_iterations,n_iterations_cg,num_workers,dtype,time,method,sketch_fn
104,2048,64,63.668717,2048,0.000122,256,32,256,0.000001,0.0001,200,1000,1,float64,0.000837,dm,none
184,2048,64,31.972399,2048,0.000488,256,32,256,0.000001,0.0001,200,1000,1,float64,0.000860,dm,none


# Sparse Cholesky decomposition

In [6]:
a, b = load_real_data(dataset_name='rcv1')
ns = 10000
ds = 200
ab = a[:ns, :ds]
bb = b[:ns]
rescale_data = True
check_reg_param = True

In [7]:
#start = time()
dm = DirectMethod(a=ab, b=bb, reg_param=1e-2, rescale_data=rescale_data, check_reg_param=check_reg_param)
dm.fit()
#print(f"within notebook: solve time = {time()-start}")
#print(f"within script: solve time = {t_}")

INFO:root:Rescaling data by max singular value: sigma_top=0.002423529515111234
INFO:root:direct method using sparse LU decomposition


In [8]:
cg = CG(a=ab, b=bb, reg_param=1e-3, rescale_data=rescale_data, check_reg_param=check_reg_param)

fit_params = {
    'tolerance': 1e-10,
    'n_iterations': 100,
    'get_full_metrics': False
}
cg.fit(**fit_params)
print(cg.metrics.keys())

INFO:root:Rescaling data by max singular value: sigma_top=0.002544724042674037


dict_keys(['average_gradient_norms', 'exit_code'])


In [145]:
start_ = time()
pcg = PCG(a=ab, b=bb, reg_param=1e-3, rescale_data=rescale_data, check_reg_param=check_reg_param)
print(f"within notebook: init time = {time()-start_}")
start = time()
x_opt, errors, residuals, times, iters_, ssizes = pcg.solve(sketch_size=1024, 
                                                            sketch_fn='sjlt', 
                                                            n_iterations=100, 
                                                            tolerance=1e-10, 
                                                            get_metrics=True)
print(f"within notebook: solve time = {time()-start}")
print(f"within script: solve time = {np.cumsum(times)[-1]}")
print(f"within notebook: total time = {time()-start_}")

INFO:root:Rescaling data by max singular value: sigma_top=0.15448754352051494
INFO:root:preconditioned conjugate gradient method: sketch_fn='sjlt', sketch_size=1024, tolerance=1e-10, n_iterations=100


within notebook: init time = 5.202043281999977


INFO:root:termination: maximum number of iterations - tolerance=1e-10, mean_err=1.5536731833112043e-05


within notebook: solve time = 34.565725970999665
within script: solve time = 24.467998789999
within notebook: total time = 39.768233457999486


In [146]:
start_ = time()
pcg = PCG(a=ab, b=bb, reg_param=1e-3, rescale_data=rescale_data, check_reg_param=check_reg_param)
print(f"within notebook: init time = {time()-start_}")
start = time()
x_opt, mean_err, exit_code = pcg.solve(sketch_size=1024, 
                                        sketch_fn='sjlt', 
                                        n_iterations=100, 
                                        tolerance=1e-10, 
                                        get_metrics=False)
print(f"within notebook: solve time = {time()-start}")
print(f"within notebook: total time = {time()-start_}")

INFO:root:Rescaling data by max singular value: sigma_top=0.1539839004410614
INFO:root:preconditioned conjugate gradient method: sketch_fn='sjlt', sketch_size=1024, tolerance=1e-10, n_iterations=100


within notebook: init time = 5.48554232099923
within notebook: solve time = 25.53425026199966
within notebook: total time = 31.02022687799945


In [4]:
ab = a[:,:10000]
ab

<804414x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 12762973 stored elements in Compressed Sparse Row format>

In [5]:
sab = sjlt(a=ab, sketch_size=2000)
#sab_t = sab.T
sab

<2000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 2725407 stored elements in Compressed Sparse Column format>

In [46]:
print(sab.nnz / (sab.shape[0]*sab.shape[1]))
print(ab.nnz / (ab.shape[0]*ab.shape[1]))

h = sab @ sab.T #+ 1e-2 * sparse_identity_matrix(sab.shape[0])

print(h.nnz / (h.shape[0]*h.shape[1]))

0.1362609
0.001586617463147086
1.0


In [ ]:
for d_ in [20000, 40000]:
    ab = a[:,:d_]
    for sketch_size in [128, 256, 512, 1024, 2048, 4096, 8192]:
        print(f"\n{d_=}, {sketch_size=}")
        method1(ab, sketch_size=sketch_size, verbose=True)
        %timeit upper_mat = method1(ab, sketch_size=sketch_size, verbose=False)
        method2(ab, sketch_size=sketch_size, verbose=True)
        %timeit upper_mat = method2(ab, sketch_size=sketch_size, verbose=False)
        method3(ab, sketch_size=sketch_size, verbose=True)
        %timeit upper_mat = method3(ab, sketch_size=sketch_size, verbose=False)
        method4(ab, sketch_size=sketch_size, verbose=True)
        %timeit upper_mat = method4(ab, sketch_size=sketch_size, verbose=False)


d_=20000, sketch_size=128

method1: sa.nnz/(sa.shape[0]*sa.shape[1])=0.447492578125
947 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

method2: sasa.nnz/(sasa.shape[0]*sasa.shape[1])=1.0
1.16 s ± 36.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

method 3: h.nnz/(h.shape[0]*h.shape[1])=1.0
1.14 s ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

method 4: h.nnz/(h.shape[0]*h.shape[1])=1.0
1.13 s ± 36.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

d_=20000, sketch_size=256

method1: sa.nnz/(sa.shape[0]*sa.shape[1])=0.3489388671875
960 ms ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

method2: sasa.nnz/(sasa.shape[0]*sasa.shape[1])=1.0
1.56 s ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

method 3: h.nnz/(h.shape[0]*h.shape[1])=1.0
1.61 s ± 70.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

method 4: h.nnz/(h.shape[0]*h.shape[1])=1.0
1.6 s ± 52.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
def method1(aa, sketch_size=2000, verbose=False):
    sa = sjlt(a=aa, sketch_size=sketch_size)
    if verbose: print(f"\nmethod1: {sa.nnz/(sa.shape[0]*sa.shape[1])=}")
    sa_d = sa.todense()
    m, d = sa_d.shape
    if m > d:
        h = sa_d.T @ sa_d + 0.1 * np.eye(d)
    else:
        h = sa_d @ sa_d.T + 0.1 * np.eye(m)
    upper_mat = np.linalg.cholesky(h)
    return upper_mat

def method2(aa, sketch_size=2000, verbose=False):
    sa = sjlt(a=aa, sketch_size=sketch_size)
    m, d = sa.shape
    if m > d:
        sasa = sa.T @ sa
        if verbose: print(f"\nmethod2: {sasa.nnz/(sasa.shape[0]*sasa.shape[1])=}")
        h = sasa.todense() + 0.1 * np.eye(d)
    else:
        sasa = sa @ sa.T
        if verbose: print(f"\nmethod2: {sasa.nnz/(sasa.shape[0]*sasa.shape[1])=}")
        h = sasa.todense() + 0.1 * np.eye(m)
    return np.linalg.cholesky(h)

def method3(aa, sketch_size=2000, verbose=False):
    sa = sjlt(a=aa, sketch_size=sketch_size)
    m, d = sa.shape
    if m > d:
        h = sa.T @ sa + 0.1 * sparse_identity_matrix(d)
    else:
        h = sa @ sa.T + 0.1 * sparse_identity_matrix(m)
    if verbose: print(f"\nmethod 3: {h.nnz/(h.shape[0]*h.shape[1])=}")
    return np.linalg.cholesky(h.todense())

def method4(aa, sketch_size=2000, verbose=False):
    sa = sjlt(a=aa, sketch_size=sketch_size)
    m, d = sa.shape
    if m > d:
        h = sa.T @ sa + 0.1 * sparse_identity_matrix(d)
    else:
        h = sa @ sa.T + 0.1 * sparse_identity_matrix(m)
    if verbose: print(f"\nmethod 4: {h.nnz/(h.shape[0]*h.shape[1])=}")
    return sparse_factorization(h.tocsc())

In [33]:
%timeit sab_t.T @ sab_t
%timeit r_mat = sparseqr.qr(sab_t.tocoo(), economy=True)[1]

r_mat = sparseqr.qr(sab_t.tocoo(), economy=True)[1]
%timeit sparse_factorization((r_mat.T.tocsr() @ r_mat.tocsc() + 0.1*sparse_identity_matrix(r_mat.shape[1])).tocsc())

%timeit sjlt(a=ab, sketch_size=2000)
%timeit sparse_factorization(sab @ sab.T + 1e-2 * sparse_identity_matrix(sab.shape[0]))
%timeit cholesky_AAt(sab, beta=1e-2)

1.99 s ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
18.6 s ± 1.87 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
597 ms ± 5.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
453 ms ± 7.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.45 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.37 s ± 229 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
z = np.random.randn(sab.shape[0])

cg_inner = CG(a=sab.T.tocsc(), b=z, reg_param=1e-1, least_squares=False, rescale_data=False, check_reg_param=False)

In [38]:
x_opt = cg_inner.solve(n_iterations=10)[0]

INFO:root:conjugate gradient method: n_iterations=10, tolerance=1e-10
INFO:root:success: CG gradient norms <= tolerance


In [39]:
x_opt

array([[ 13.7714368 ],
       [225.5460754 ],
       [ 18.27657872],
       ...,
       [ -3.69530684],
       [ 46.21919365],
       [ 60.88713896]])

In [24]:
%timeit r_mat = sparseqr.qr(sab_t, economy=True)[1]

336 ms ± 2.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit sparse_factorization((r_mat.T.tocsr() @ r_mat.tocsc() + 0.1*sparse_identity_matrix(r_mat.shape[1])).tocsc())

552 ms ± 9.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit sab_t.T @ sab_t

18 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
type(sab.todense())

numpy.matrix